In [1]:
%load_ext autoreload
%autoreload 2

# Localization by specific scene elements (Semantic Object Context (SOC) module)

A module that implements a neural network algorithm to search a database of places already visited by a car for the most similar records using the car's "semantic object context" - the number and position of certain scene elements (e.g., doors, road signs, traffic signs, and so on).

In [21]:
from time import time

import faiss

from tqdm import tqdm

from hydra.utils import instantiate
import numpy as np
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
import torch
from torch.utils.data import DataLoader

from opr.datasets.itlp import ITLPCampus
from opr.pipelines.place_recognition import PlaceRecognitionPipeline

In [22]:
DATABASE_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/indoor/00_2023-10-25-night"
QUERY_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/indoor/01_2023-11-09-twilight"

SENSOR_SUITE = []

BATCH_SIZE = 64
NUM_WORKERS = 4
DEVICE = "cuda"

SOC_WEIGHTS_PATH = "../weights/place_recognition/soc.pth"
MODEL_CONFIG_PATH = "../configs/model/place_recognition/multi-image_multi-semantic_lidar_late-fusion.yaml"
WEIGHTS_PATH = "../weights/place_recognition/multi-image_multi-semantic_lidar_late-fusion_nclt.pth"

DATASET_CONFIG_PATH = "../../configs/dataset/itlp.yaml"

In [23]:
def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error

## Prepare faiss database index

In [24]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../../configs"):
    cfg = compose(config_name="train_soc")

db_dataset_cfg = cfg.dataset
db_dataset_cfg.dataset_root = DATABASE_TRACK_DIR
db_dataset_cfg.csv_file = "track.csv"
db_dataset_cfg.sensors = SENSOR_SUITE
db_dataset_cfg.load_semantics = False
db_dataset_cfg.test_split = [2]
db_dataset = instantiate(db_dataset_cfg, subset="test")
# ITLPCampus(
#     dataset_root=DATABASE_TRACK_DIR,
#     sensors=["front_cam", "back_cam", "lidar"],
#     mink_quantization_size=0.5,
#     load_semantics=True,
#     subset="test",
#     test_split=[2, 4],
# )

In [25]:
db_dataset.dataset_df

,index,floor,timestamp,front_cam_ts,back_cam_ts,lidar_ts,tx,ty,tz,qx,qy,qz,qw,in_query
0,241,2,1698264504041122048,1698264504002182709,1698264504047208484,1698264504047677052,0.936002,-0.005618,99.978601,-0.002076,0.001280,-0.068510,0.997647,True
1,242,2,1698264507070846464,1698264507119317501,1698264507057790564,1698264507073838801,1.912760,0.557575,100.056236,-0.002415,0.004182,-0.037406,0.999288,True
2,243,2,1698264508581344000,1698264508585594131,1698264508578256338,1698264508589850480,2.830138,0.239738,100.065229,-0.003375,-0.008576,-0.475835,0.879486,True
3,244,2,1698264509993314304,1698264510033376489,1698264510019705506,1698264509997810962,3.150359,-0.712376,100.126280,0.000956,-0.018858,-0.707919,0.706042,True
4,245,2,1698264511203823360,1698264511217856467,1698264511206572579,1698264511206429225,3.112751,-1.740502,100.169964,0.008021,-0.017280,-0.785810,0.618174,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,491,2,1698264849718639616,1698264849676965759,1698264849689958331,1698264849722924313,2.660992,2.294300,99.866778,-0.024562,0.022925,0.607234,-0.793813,True
251,492,2,1698264851028064512,1698264850990813971,1698264850985596943,1698264851032721570,2.853010,1.271028,99.926578,-0.018994,0.013818,0.769123,-0.638669,True
252,493,2,1698264852642632704,1698264852640794484,1698264852612610043,1698264852646716773,2.486120,0.393792,99.996949,-0.022611,0.017190,0.975104,-0.219922,True
253,494,2,1698264854963774720,1698264854908024867,1698264854953993503,1698264854969959146,1.486615,0.207877,100.089107,-0.022693,0.022959,0.999041,0.029591,True


In [26]:
db_dataloader = DataLoader(
    db_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=db_dataset.collate_fn,
)


In [27]:
model_config = cfg.model
model = instantiate(model_config)
model_weights = torch.load(SOC_WEIGHTS_PATH)["model_state_dict"]
model.load_state_dict(model_weights, strict=True)
model = model.to(DEVICE)
model.eval();
model

SOCMLPMixer(
  (mlp_mixer): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=1, p2=1)
    (1): Linear(in_features=30, out_features=64, bias=True)
    (2): Sequential(
      (0): PreNormResidual(
        (fn): Sequential(
          (0): Conv1d(72, 288, kernel_size=(1,), stride=(1,))
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Conv1d(288, 72, kernel_size=(1,), stride=(1,))
          (4): Dropout(p=0.0, inplace=False)
        )
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (1): PreNormResidual(
        (fn): Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=32, out_features=64, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (3

In [28]:
descriptors = []
with torch.no_grad():
    for batch in tqdm(db_dataloader):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        final_descriptor = model(batch)["final_descriptor"]
        descriptors.append(final_descriptor.detach().cpu().numpy())

descriptors = np.concatenate(descriptors, axis=0)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:20<00:00,  5.18s/it]


In [29]:
print(descriptors.shape)

(255, 256)


In [30]:
index = faiss.IndexFlatL2(descriptors.shape[1])
index.add(descriptors)

In [31]:
faiss.write_index(index, DATABASE_TRACK_DIR + "/index.faiss")


## Test

In [32]:
pipe = PlaceRecognitionPipeline(
    database_dir=DATABASE_TRACK_DIR,
    model=model,
    model_weights_path=None,
    device=DEVICE,
)

pipe.database_df = db_dataset.dataset_df  # !!!!!

In [33]:
query_dataset_cfg = cfg.dataset
query_dataset_cfg.dataset_root = QUERY_TRACK_DIR
query_dataset_cfg.csv_file = "track.csv"
query_dataset_cfg.sensors = SENSOR_SUITE
query_dataset_cfg.load_semantics = False
query_dataset_cfg.test_split = [2]
query_dataset = instantiate(query_dataset_cfg, subset="test")

# query_dataset = ITLPCampus(
#     dataset_root=QUERY_TRACK_DIR,
#     sensors=SENSOR_SUITE,
#     mink_quantization_size=0.5,
#     load_semantics=True,
# )


In [34]:
query_dataset.dataset_df


,index,floor,timestamp,front_cam_ts,back_cam_ts,lidar_ts,tx,ty,tz,qx,qy,qz,qw,in_query
0,239,2,1699538667594320384,1699538667595033775,1699538667576960360,1699538667597536130,0.178382,0.067620,100.008046,0.001788,0.002456,0.056518,0.998397,True
1,240,2,1699538671628715008,1699538671601782558,1699538671650855565,1699538671631259615,1.178769,0.125520,100.219715,0.006708,0.076623,0.006842,0.997014,True
2,241,2,1699538675763979776,1699538675794472055,1699538675788658434,1699538675766731756,2.128260,0.192668,100.084871,0.009729,0.004442,0.004434,0.999933,True
3,242,2,1699538676974509824,1699538676993606473,1699538676990979660,1699538676978077620,3.114585,0.151316,100.129671,0.013395,0.006937,-0.371526,0.928300,True
4,243,2,1699538678386366208,1699538678393822673,1699538678385266571,1699538678388709906,3.391258,-0.801008,100.086859,0.017389,-0.006268,-0.712724,0.701201,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,483,2,1699539033314420736,1699539033299984007,1699539033337416186,1699539033317537460,3.003310,1.641420,100.043739,-0.000129,-0.003920,0.753434,-0.657512,True
245,484,2,1699539035533363456,1699539035502073052,1699539035519003859,1699539035535765114,2.542848,0.774170,100.231331,0.011742,0.015746,0.968250,-0.249211,True
246,485,2,1699539038054853888,1699539038097976464,1699539038063191280,1699539038057209443,1.579236,0.492064,100.151714,-0.011468,0.016086,0.997928,-0.061235,True
247,486,2,1699539041887878912,1699539041908761612,1699539041857468480,1699539041895371948,0.582539,0.375259,100.158292,-0.057342,0.011669,0.997251,-0.045464,True


In [35]:
PR_THRESHOLD = 25.0

pr_matches = []
dist_errors = []
angle_errors = []
times = []

for query in tqdm(query_dataset):
    t = time()
    output = pipe.infer(query)
    times.append(time() - t)
    dist_error, angle_error = compute_error(output["pose"], query["pose"])
    print(f"dist_error: {dist_error}, angle_error: {angle_error}")
    pr_matches.append(dist_error < PR_THRESHOLD)
    dist_errors.append(dist_error)
    angle_errors.append(angle_error)

times = times[1:]  # the first query is always slower

  0%|          | 1/249 [00:00<00:31,  7.81it/s]

  1%|          | 3/249 [00:00<00:25,  9.50it/s]

dist_error: 2.657951059981733, angle_error: 63.32476580736517
dist_error: 12.565176700529403, angle_error: 86.24313498093596
dist_error: 11.96044346985411, angle_error: 41.24458042259357


  2%|▏         | 5/249 [00:00<00:26,  9.22it/s]

dist_error: 10.98697491257253, angle_error: 85.07475687119938
dist_error: 11.13970395570753, angle_error: 76.66339471664031
dist_error: 0.10329121167224774, angle_error: 20.700984771538202


...

dist_error: 13.271983491726884, angle_error: 72.49392317770979
dist_error: 0.4065259622647763, angle_error: 3.663105221028232
dist_error: 14.94800496016716, angle_error: 88.78158764665436


 99%|█████████▉| 246/249 [00:25<00:00, 10.48it/s]

dist_error: 10.173930206502298, angle_error: 33.47339253731454
dist_error: 19.545878966940943, angle_error: 6.715180769977934


100%|██████████| 249/249 [00:25<00:00,  9.65it/s]

dist_error: 0.1588953360651471, angle_error: 8.298033957669276
dist_error: 6.190492218639788, angle_error: 16.012994370544362


In [36]:
print(f"Recall@1: {(np.mean(pr_matches))*100:.2f}")
print(f"Mean distance error: {np.mean(dist_errors):.2f}, mean angle error: {np.mean(angle_errors):.2f}")
print(f"Median distance error: {np.median(dist_errors):.2f}, median angle error: {np.median(angle_errors):.2f}")

Recall@1: 83.13
Mean distance error: 12.38, mean angle error: 15.75
Median distance error: 4.88, median angle error: 6.38


In [37]:
print(f"Mean inference time: {np.mean(times)*1000:.2f} ms, median inference time: {np.median(times)*1000:.2f} ms")

Mean inference time: 1.98 ms, median inference time: 1.96 ms
